In [1]:
import sys
sys.path.append("../")
import numpy as np 
from utils import *
from lattice import *
import pandas
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensornetwork as tn

In [2]:
sps = 3
bd = 2
seed = 36
A = FF.block1D(bd, sps, bd, seed=seed, canonical=True).real
tmp = np.zeros((2,2))
for i in range(2):
    tmp += A[:,i,:] @ A[:,i,:].T
print(tmp)

[[ 0.93465598 -0.22428086]
 [-0.22428086  0.12754249]]


In [3]:
A2 = np.einsum("ijk,klm->jlim", A, A).reshape(sps**2, bd**2)
U, s , V = np.linalg.svd(A2)
Up = U[:, len(s):]
h = Up @ Up.T

In [4]:
mps_nodes, mps_edges = FF.create_MPS(6, A)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y1 = C.tensor.reshape(-1)
y1 /= np.linalg.norm(y1)

In [5]:
y_ = y1.reshape((sps**2, -1))
rho = y_ @ y_.T 
print("y is eigen state for h? : ",np.trace(rho @ h))

y is eigen state for h? :  9.312774901445156e-18


### Check canonical give same state

In [6]:
B = FF.block1D(bd, sps, bd, seed=seed, canonical=False).real
B2 = np.einsum("ijk,klm->jlim", B, B).reshape(sps**2, bd**2)
U, s , V = np.linalg.svd(B2)
Up = U[:, len(s):]
hb = Up @ Up.T
mps_nodes, mps_edges = FF.create_MPS(6, B)
for k in range(len(mps_edges)):
    C = tn.contract(mps_edges[k])
y2 = C.tensor.reshape(-1)
y2 /= np.linalg.norm(y2)
np.linalg.norm(y1 - y2), np.linalg.norm(h - hb)

(5.114424018633868e-16, 2.3865583919308875e-15)

### Check energy gap

In [30]:
TA = np.einsum("jik,lim->jlkm", A, A).reshape(bd**2, bd**2)
e_ = np.sort(np.linalg.eigvals(TA))
gap_transfer = e_[-1] - e_[-2]
gaps = []
for L in range(2, 8):
    H = sum_ham(h, [[i, (i + 1) % L] for i in range(L)], L, sps)
    E = np.linalg.eigvalsh(H)
    gap = E[1] - E[0]
    gaps.append(gap)

ValueError: operand has more dimensions than subscripts given in einstein sum, but no '...' ellipsis provided to broadcast the extra dimensions.

### Create MPS from transfer matrix

```
k - | A^T | - l
i - | A | - j
```

should be symmetric to $ i \leftrightarrow j, k \leftrightarrow l $ ( vertical flip) and horizonal flip 

In [142]:
bd = 2
A0 = np.random.normal(size=(bd**2, bd**2))#(i,k) x (j,l)  matrix 
A = A0.reshape(bd,bd,bd,bd) 

In [143]:
A = (A.transpose((1,0,3,2)) + A.transpose((2,3,0,1)) + A.transpose((3,2,1,0)) + A) / 4

- Next, bundle $(k, l)$ and $(i, j)$
- Make sure B is symmetric matrix

In [135]:
B = np.einsum("ijkl->ikjl",A).reshape(bd**2, bd**2)
print(B)

[[ 5.61098505  1.18567413  1.18567413  4.58090475]
 [ 1.18567413 -0.04670784  0.08688897 -0.68035489]
 [ 1.18567413  0.08688897 -0.04670784 -0.68035489]
 [ 4.58090475 -0.68035489 -0.68035489  4.25721818]]


In [145]:
U, S, V = np.linalg.svd(B)

In [161]:
U 

array([[-7.64016249e-01, -4.37717500e-01,  4.74006921e-01,
        -1.21754692e-16],
       [-4.90595072e-02, -4.77065277e-01, -5.19617057e-01,
         7.07106781e-01],
       [-4.90595072e-02, -4.77065277e-01, -5.19617057e-01,
        -7.07106781e-01],
       [-6.41455766e-01,  5.94323845e-01, -4.85091401e-01,
         6.10820386e-17]])

In [162]:
V.T

array([[-7.64016249e-01, -4.37717500e-01, -4.74006921e-01,
         0.00000000e+00],
       [-4.90595072e-02, -4.77065277e-01,  5.19617057e-01,
        -7.07106781e-01],
       [-4.90595072e-02, -4.77065277e-01,  5.19617057e-01,
         7.07106781e-01],
       [-6.41455766e-01,  5.94323845e-01,  4.85091401e-01,
        -9.61483260e-17]])

In [132]:
E, V = np.linalg.eigh(B)
np.linalg.norm(V @ np.diag(E) @ V.T-B), E

(5.338100145117183e-15,
 array([-0.30370736,  0.36945071,  0.74403308,  7.73287618]))

```
k - | A^T  A = B | - i
l - |            | - j
```

```
k - | db^T |  -  | db | - 
l - |      |     |    | -  
```

In [127]:
db = np.diag(np.sqrt(E)) @ V.T
np.linalg.norm(db.T @ db - B)

/var/folders/z7/m5qzz0hj12z06hxsn5km2c400000gn/T/ipykernel_3674/1316602744.py:1: RuntimeWarning: invalid value encountered in sqrt
  db = np.diag(np.sqrt(E)) @ V.T


nan

### invariant under reflective symmetry

```   
      py
       |
i - | da | -   j
```

In [128]:
A_MPS = db.reshape(4, 2, 2)
np.linalg.norm(A_MPS - A_MPS.transpose(0, 2, 1))

nan

### Reconstruct transition matrix

In [129]:
A_tilde_ = np.einsum("pij,pkl->ikjl", A_MPS, A_MPS)
A_tilde = A_tilde_.reshape(4,4)

In [130]:
np.linalg.norm(A_tilde - A0)

nan

In [131]:
A_tilde_ - A

array([[[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]],


       [[[nan, nan],
         [nan, nan]],

        [[nan, nan],
         [nan, nan]]]])